<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/assignment4/B)_1_basic_Lenet(mnist).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graded Assignment 4 : CNNs

b) CNN basics : use pytorch and from scratch (not using existing architecture shipped models)  write basic LeNet (mnist) and VGG-16 model (cifar100) and and RESNET (only 8 layers - not 52)  architectures - run on gpu  (use .cuda instruction) - train and report all the stats

# B) 1_basic Lenet(mnist)

In [0]:
import argparse
import torch
from torch import nn,optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.nn import Module,Conv2d,Dropout2d,Linear

# Network definition

In [0]:
class LeNet(Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = Conv2d(1, 32, 3, 1)
        self.conv2 = Conv2d(32, 64, 3, 1)
        self.dropout1 = Dropout2d(0.25)
        self.dropout2 = Dropout2d(0.5)
        self.fc1 = Linear(9216, 128)
        self.fc2 = Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## load data (use GPU)

In [7]:
torch.manual_seed(42)
use_cuda = torch.cuda.is_available()
print(use_cuda)

device = torch.device("cuda" if use_cuda else "cpu")

device

True


device(type='cuda')

In [0]:
args = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.MNIST
                   ('../data', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
                batch_size=batch_size, shuffle=True, **args)


test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
               transform=transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])),
                batch_size=test_batch_size, shuffle=True, **args)



###Fit model

In [0]:
batch_size = 64
test_batch_size = 1000
epochs = 10
gamma = 0.7
lr = 0.01
log_interval = 100

In [0]:
model = LeNet().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [34]:
!pip install tensorboardX

     |████████████████████████████████| 204kB 4.6MB/s 


In [0]:
import torchvision
from torch.autograd import Variable
from tensorboardX import SummaryWriter

# Training 

In [25]:
pip install tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter 
log_dir="logs/LENET-Pytorch-mnist"       
writer = SummaryWriter(log_dir)

In [30]:
tensorboard --logdir=/log_dir/ --port 6006

ERROR: Failed to launch TensorBoard (exited with 255).
Contents of stderr:
2020-05-18 23:22:56.368914: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
E0518 23:22:57.961052 140140586674048 program.py:311] TensorBoard could not bind to port 6006, it was already in use
ERROR: TensorBoard could not bind to port 6006, it was already in use

In [0]:
def training(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format
                  (epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))




In [24]:
for epoch in range(1, epochs + 1):
   training(log_interval, model, device, train_loader, optimizer, epoch)   
   scheduler.step()

   torch.save(model.state_dict(), "mnist_cnn_lenet_pytorch.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.413098
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.337891
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.244960
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.336138
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.173782
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.281404
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.315838
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.172990
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.221431
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.440308
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.313301
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.247516
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.300868
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.130253
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.324525
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.384089
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.191285
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.179453
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.218266
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.207432
Train Epoch:

## Testing

In [0]:
def testing(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest dataset: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [48]:
testing(model, device, test_loader)


Test dataset: Average loss: 0.1715, Accuracy: 9497/10000 (95%)



In [0]:
log_dir="logs/LENET-Pytorch-mnist"   
writer = SummaryWriter(log_dir=log_dir, comment='LENET-Pytorch-mnist')
writer.add_graph(model,train_loader)

In [0]:
%load_ext tensorboard

%tensorboard --logdir logs